# OECD data

## Notes

### SSL issues

From January 2023 I got the error message: SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED. 
I gather the immediate issue is changes to the openSSL library. But the more significant issue is with the OECD website.

To fix this issue, I:

* created a text file named ~/ssl.conf with the following content:

        openssl_conf = openssl_init
    
        [openssl_init]
        ssl_conf = ssl_sect
    
        [ssl_sect]
        system_default = system_default_sect

        [system_default_sect]
        Options = UnsafeLegacyRenegotiation

* added this line to my .zshrc file (because I use zsh as my shell): 

        export OPENSSL_CONF=~/ssl.conf

## Python setup

### Imports

In [1]:
# system imports
import sys
from operator import ge, gt, le, lt
from pathlib import Path

# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# ABS
from abs_data_capture import (
    find_rows,
    get_ABS_meta_and_data,
    get_fs_constants,
    get_meta_constants,
    get_plot_constants,
    iudts_from_row,
)
from bis_data_capture import get_bis_cbpr
from henderson import hma
from matplotlib.lines import Line2D
from pandasdmx import Request

# local imports
from plotting import finalise_plot, set_chart_dir
from requests.exceptions import HTTPError

### Settings

In [2]:
# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# plotting stuff
plt.style.use("fivethirtyeight")

# multi-time-period charts
TODAY = pd.Timestamp("today")
RECENCY_PERIOD = 5  # years
MONTH_ADJ = 2  # months
RECENT = TODAY - pd.DateOffset(years=RECENCY_PERIOD, months=MONTH_ADJ)

# Other
SOURCE = "Source: OECD"

# Where to put the charts
CHART_DIR = "./charts/OECD"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
set_chart_dir(CHART_DIR)

# Remove old charts
for filename in Path(CHART_DIR).glob("*.png"):
    filename.unlink()

# display charts in notebook
SHOW = False

## OECD support

### Data

In [3]:
location_map = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "CHL": "Chile",
    "CZE": "Czech Rep.",
    "DNK": "Denmark",
    "EST": "Estonia",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "HUN": "Hungary",
    "ISL": "Iceland",
    "IRL": "Ireland",
    "ISR": "Israel",
    "ITA": "Italy",
    "JPN": "Japan",
    "KOR": "Korea",
    "LVA": "Latvia",
    "LUX": "Luxembourg",
    "MEX": "Mexico",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "POL": "Poland",
    "PRT": "Portugal",
    "SVK": "Slovak Rep.",
    "SVN": "Slovenia",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "TUR": "Turkey",
    "GBR": "United Kingdom",
    "USA": "United States",
    "ARG": "Argentina",
    "BRA": "Brazil",
    "CHN": "China",
    "COL": "Colombia",
    "CRI": "Costa Rica",
    "IND": "India",
    "IDN": "Indonesia",
    "LTU": "Lithuania",
    "RUS": "Russia",
    "SAU": "Saudi Arabia",
    "ZAF": "South Africa",
}

In [4]:
chart_sets = {
    # Limit charts to 6 nations - otherwise they are too cluttered
    "of_interest": [
        "AUS",
        "USA",
        "CAN",
        "DEU",
        "GBR",
        "JPN",
    ],
    "anglosphere": [
        "AUS",
        "USA",
        "CAN",
        "NZL",
        "GBR",
        "IRL",
    ],
    "major_europe": [
        "FRA",
        "DEU",
        "ITA",
        "GBR",
        "RUS",
        "ESP",
    ],
    "largest_economies": [
        "USA",
        "CHN",
        "JPN",
        "DEU",
        "GBR",
        "IND",
    ],
    "asia": [
        "KOR",
        "JPN",
        "CHN",
        "IND",
        "IDN",
    ],
    "north_europe": [
        "DNK",
        "SWE",
        "NOR",
        "ISL",
        "FIN",
        "GBR",
    ],
    "baltic_europe": [
        "LVA",
        "LTU",
        "EST",
    ],
    "central_europe": [
        "CZE",
        "HUN",
        "SVK",
        "SVN",
        "POL",
        "GRC",
    ],
    "west_europe": [
        "ESP",
        "PRT",
        "NLD",
        "LUX",
        "FRA",
        "BEL",
    ],
    "italo_germanic_europe": [
        "DEU",
        "AUT",
        "CHE",
        "ITA",
    ],
    "turkey": [
        "TUR",
    ],  # a high inflation outlier.
    "n_america": [
        "USA",
        "CAN",
        "MEX",
    ],
    "c_s_america": [
        "CHL",
        "ARG",
        "BRA",
        "COL",
        "CRI",
    ],
    "other": [
        "AUS",
        "NZL",
        "SAU",
        "ZAF",
        "ISR",
    ],
}

### Python functions

In [5]:
def build_key(locations, subject, measure, frequency, start):
    if frequency:
        key = [locations, subject, measure, frequency]
    else:
        key = [locations, subject, measure]
    key_string = f"{'.'.join(['+'.join(x) for x in key])}" f"/all?startTime={start}"
    return key_string

In [6]:
# get the raw index data
def get_oecd_data(resource_id, key_string):
    oecd = Request("OECD")

    try:
        data = oecd.data(
            resource_id=resource_id,
            key=key_string,
        ).to_pandas()

    except HTTPError as err:
        print(f"HTTPError: {err}")
        data = pd.DataFrame()

    return data

In [7]:
def download_qtrly_data(resource_id, key_string):
    data = (
        get_oecd_data(resource_id=resource_id, key_string=key)
        .unstack()
        .T.dropna(how="all", axis="rows")
        .droplevel(axis=1, level=[1, 2, 3])
    )
    data.index = pd.PeriodIndex(data.index, freq="Q")

    return data

In [8]:
def annualise_percentages(series: pd.Series, periods: int = 12) -> pd.Series:
    """Annualise a growth rate for a period."""

    s = series / 100.0
    annual = (((1 + s) ** periods) - 1) * 100
    return annual

## OECD GDP data

### Real GDP growth

In [9]:
# rGDP - get the data
resource_id = "QNA"  # Quarterly National Accounts
measure = "VPVOBARSA"  # US dollars, volume estimates, fixed PPPs,
# OECD reference year, annual levels,
# seasonally adjusted
subject = "B1_GE"  # Gross domestic product - expenditure approach
rfooter = f"Source: OECD {resource_id} {subject} {measure}"
lfooter = "Volume est., US$, fixed PPPs, Seas Adj"

key = build_key(location_map, [subject], [measure], ["Q"], "1959")
rgdp = download_qtrly_data(resource_id, key)
rgdp_growth = rgdp.pct_change(1, fill_method=None) * 100

In [10]:
# diagnostics
rgdp_growth.shape

(256, 45)

In [11]:
# check for missing data in the final period ...
final_row = rgdp.iloc[-1]
missing_count = final_row.isna().sum()
if missing_count:
    print(f"Final period: {final_row.name}")
    print(f"Missing data count for final period: {missing_count}")
    print(f"Missing data belongs to: {rgdp.columns[final_row.isna()].to_list()}")
    print(f"Nations with final data: {rgdp.columns[final_row.notna()].to_list()}")

Final period: 2022Q4
Missing data count for final period: 1
Missing data belongs to: ['RUS']
Nations with final data: ['ARG', 'AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'CHL', 'COL', 'CRI', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HUN', 'IDN', 'IND', 'IRL', 'ISL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX', 'NLD', 'NOR', 'NZL', 'POL', 'PRT', 'SAU', 'SVK', 'SVN', 'SWE', 'TUR', 'USA', 'ZAF']


In [12]:
# GDP identify/count quarterly contractions
contractions = rgdp_growth < 0
contraction_count = contractions.sum(axis=1)

start = pd.Period("1999Q1", freq="Q")
title = "Num. OECD Monitored States with Qrtly GDP contraction"
ax = contraction_count[contraction_count.index >= start].plot.bar()
ax.set_xticks(ax.get_xticks()[::4])
finalise_plot(
    ax,
    title=title,
    ylabel="Count",
    rfooter=rfooter,
    # show=True,
)

In [13]:
# List nations in contraction at last quarter
", ".join([location_map[x] for x in contractions.iloc[-1][contractions.iloc[-1]].index])

'Argentina, Austria, Brazil, Czech Rep., Germany, Estonia, Finland, Hungary, Italy, Korea, Lithuania, Luxembourg, Latvia, New Zealand, Poland, Sweden, South Africa'

In [14]:
# GDP identify/count recessions
recession = (rgdp_growth < 0) & (rgdp_growth.shift(1) < 0)
recession_count = recession.sum(axis=1)

title = "Number of OECD Monitored States in Recession"
ax = recession_count[recession_count.index >= start].plot.bar()
ax.set_xticks(ax.get_xticks()[::4])
finalise_plot(
    ax,
    title=title,
    ylabel="Count",
    rfooter=rfooter,
    lfooter="Recession defined as two quarters of negative GDP growth",
    show=SHOW,
)

In [15]:
# List nations in recession at last quarter
", ".join([location_map[x] for x in recession.iloc[-1][recession.iloc[-1]].index])

'Czech Rep., Estonia, Finland, Hungary, Latvia'

In [16]:
if False:
    # GDP growth plots
    title = "GDP Growth: quarter on quarter"
    for label, locations in chart_sets.items():
        # get the data
        loc = [x for x in locations if x in rgdp_growth.columns]
        data = rgdp_growth[loc].rename(location_map, axis=1).sort_index(axis=1)

        # plot the data
        for start in None, RECENT:
            subset = (
                data
                if start is None
                else data[data.index >= pd.Period(start, freq="Q")]
            )
            ax = subset.plot(lw=2.5)
            ax.legend(title=None, loc="best", ncol=2)
            tag = label if start is None else f"{label}-recent"

            finalise_plot(
                ax,
                title=title,
                ylabel="Per cent",
                tag=tag,
                rfooter=rfooter,
                lfooter=lfooter,
                zero_y=True,
                show=SHOW,
            )

### Nominal GDP - national currency

In [17]:
# nGDP - get the data
resource_id = "QNA"  # Quarterly National Accounts
measure = "CQRSA"  # National currency, current prices, quarterly levels
subject = "B1_GE"  # Gross domestic product - expenditure approach
rfooter = f"Source: OECD {resource_id} {subject} {measure}"

key = build_key(location_map, [subject], [measure], ["Q"], "2000")
nGDP = download_qtrly_data(resource_id, key)
nGDP.tail()

LOCATION,ARG,AUS,AUT,BEL,BRA,CAN,CHE,CHL,COL,CRI,CZE,DEU,DNK,ESP,EST,FIN,FRA,GBR,GRC,HUN,IDN,IND,IRL,ISL,ISR,ITA,JPN,KOR,LTU,LUX,LVA,MEX,NLD,NOR,NZL,POL,PRT,RUS,SVK,SVN,SWE,TUR,USA,ZAF
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021Q4,13697133.8,563583.0,104708.8,130490.0,2289899.5,655544.0,187055.6,63850138.3,325998174.2,10449180.2,1552687.0,924492.0,660807.3,319346.0,8406.0,64685.0,642048.5,592735.0,48075.8,14851602.0,4498991.7,62200.7,110218.4,882291.1,417439.0,458729.5,137870025.0,530268900.0,14917.7,19073.7,8783.0,6629022.3,220582.1,1214898.0,90220.0,692051.1,55793.7,NaN,25403.6,13775.1,1425259.0,2196859.1,6087280.3,1571828.5
2022Q1,15757783.7,587758.0,107690.6,134283.0,2353631.3,679488.0,190168.5,64260284.6,340969691.1,10700626.9,1639333.0,944141.0,669438.5,320214.0,8647.8,65097.0,646767.9,606267.0,50375.1,15589297.0,4592686.2,63905.4,117758.4,903656.1,424716.0,465654.4,138208150.0,532765700.0,15816.4,19315.1,9390.3,6845736.9,226141.7,1319765.0,91369.0,726572.6,57873.2,NaN,25964.2,14126.1,1429647.0,2805634.7,6185120.0,1610802.4
2022Q2,18652185.0,611328.0,111505.0,136264.0,2471969.0,706412.0,192239.4,64372679.5,364158130.1,10987919.4,1678001.0,956182.0,702720.9,328500.0,8946.6,66420.0,654785.2,618210.0,51603.9,16307588.0,4890042.2,65524.4,125003.6,912239.2,435834.0,476329.7,139654800.0,540784300.0,16501.8,19646.9,9501.9,7075979.6,232926.0,1360103.0,94436.0,753450.0,59405.5,NaN,26663.7,14559.0,1473667.0,3573037.4,6312119.0,1653807.4
2022Q3,22249083.8,618436.0,113590.1,138915.0,2523905.0,701981.0,193703.2,66247375.1,376483118.9,11227795.9,1741502.0,969652.0,715378.4,333836.0,9124.3,67414.0,664448.9,627145.0,52027.9,17098204.0,4995205.6,67252.3,129182.0,975061.8,442352.0,476697.8,138537725.0,538803800.0,17266.3,19899.7,9829.6,7229799.7,238449.3,1496692.0,97480.0,783492.6,60205.7,NaN,27250.6,14915.2,1500693.0,4065474.7,6430985.3,1694232.0
2022Q4,25991187.3,631402.0,114785.7,139994.0,2565811.2,697238.0,194749.1,67552709.9,382262120.5,11324295.5,1734730.0,990436.0,710400.4,344558.0,9401.3,67744.0,675691.0,639616.0,53172.0,17620789.0,5110511.5,69046.3,130641.0,975458.4,451688.0,491285.1,140151825.0,539324400.0,17402.6,19295.8,10204.5,7322605.5,243477.8,1363368.0,97032.0,810676.5,61768.8,NaN,27851.6,15317.0,1510627.0,4562427.4,6534498.0,1679503.8


## OECD inflation data

### Functions

In [18]:
# download the CPI data - original series data - not seasonally adjusted ...
# becuase some nations do not seasonally adjust their data ...
def download_CPI_data(
    locations=location_map,
    start="1969",
    subject_tuple=("CP18ALTT", "CPALTT01"),
    measure="IXOB",
):
    combined = pd.DataFrame()

    q_locations = {
        "AUS",
        "NZL",
    }
    monthly = [x for x in locations if x not in q_locations]
    quarterly = [x for x in locations if x in q_locations]

    for locations, freq in zip((monthly, quarterly), ("M", "Q")):
        for subject in subject_tuple:
            if not len(locations):
                continue
            key = build_key(locations, [subject], [measure], freq, start)
            data = get_oecd_data(resource_id="PRICES_CPI", key_string=key)
            if not len(data):
                continue
            data = data.unstack().T.droplevel(axis=1, level=[1, 2, 3])
            data.index = pd.PeriodIndex(data.index, freq=freq)
            if freq == "Q":
                data = data.resample("M", convention="end").sum()
            combined = (
                data
                if not len(combined)
                else pd.concat([combined, data], axis=1, join="outer")
            ).sort_index()
            locations = [x for x in locations if x not in data.columns]

    # ignore last row when less than N data items
    N = 6
    combined = combined.dropna(thresh=N)
    combined = combined.rename(columns=location_map)

    return combined

### Calculate inflation rates

In [19]:
cpi = download_CPI_data()

HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP18ALTT.IXOB.Q/all?startTime=1969


In [20]:
# List of nations not reporting for the latest period
latest_period = cpi.index[-1]
print(f"Latest period: {latest_period}")
print(
    f"Missing CPI print: {', '.join(cpi.iloc[-1][cpi.iloc[-1].isna()].index.to_list())}"
)

Latest period: 2023-03
Missing CPI print: Japan, India, Russia, Australia


In [21]:
# raw inflation data
annual_inflation_raw = cpi.pct_change(12, fill_method=None) * 100
# The next step managers Au and NZ because they only have quarterly CPI data
annual_inflation = annual_inflation_raw.interpolate(method="time", limit_area="inside")
qrtly_inflation_raw = cpi.pct_change(3, fill_method=None) * 100
qrtly_inflation_annualised = annualise_percentages(qrtly_inflation_raw, 4).interpolate(
    method="time", limit_area="inside"
)

# smoothed / trend data
HENDERSON_TERM = 13
annual_inflation_smooth = pd.DataFrame(index=annual_inflation.index)
qrtly_inflation_annualised_smooth = pd.DataFrame(index=qrtly_inflation_annualised.index)
for nation in qrtly_inflation_annualised.columns:
    annual_inflation_smooth[nation] = hma(
        annual_inflation[nation].dropna(), HENDERSON_TERM
    )
    qrtly_inflation_annualised_smooth[nation] = hma(
        qrtly_inflation_annualised[nation].dropna(), HENDERSON_TERM
    )

# some useful constants
START_COVID = pd.Period("2020-01-01", freq="M")
HIGH_COVID = pd.Period("2021-01-01", freq="M")
linestyle = {
    "style": [
        "--.",
        "-.",
        "-",
        "--",
        ":",
    ]
    * 6
}
EXCLUDE = ["Turkey", "Russia"]  # Turkey has rampant inflation
# Russie not updating data during war

# a quick look at Australia
qrtly_inflation_annualised["Australia"].tail(9)

TIME_PERIOD
2022-07    7.361861
2022-08    7.429747
2022-09    7.497634
2022-10    7.561468
2022-11    7.625303
2022-12    7.689137
2023-01         NaN
2023-02         NaN
2023-03         NaN
Freq: M, Name: Australia, dtype: float64

### CPI over the past year

In [22]:
def get_recent_ohlc(data: pd.DataFrame) -> pd.DataFrame:
    """For a dataset, build a table of Open, Highm Low, Close
    points for last valid 13 months in each column."""

    # compare progress over 13 months because Australia
    # and New Zealand only collect CPI measures quarterly
    INCLUSIVE_YEAR = 13  # months
    index = ["Open", "High", "Low", "Close"]
    summary = pd.DataFrame([], index=index)  # return vehicle
    for name in data.columns:
        if name in EXCLUDE:
            continue
        column = data[name]
        last_valid = column.last_valid_index()
        year = pd.period_range(end=last_valid, periods=INCLUSIVE_YEAR)
        frame = column[year]
        open = frame.iloc[0]
        high = frame.max()
        low = frame.min()
        close = frame.iloc[-1]
        key = f"{name} {str(last_valid.year)[2:]}-{last_valid.month:02d}"
        summary[key] = pd.Series([open, high, low, close], index=index)
    summary = summary.T.sort_values("Close")
    return summary

In [23]:
def plot_ohlc(ohlc_df: pd.DataFrame, horizontal: bool = True, **kwargs) -> None:
    """Plot data in ohlc_df in a open-high-low-close style."""

    def xy(x, y):
        return (x, y) if horizontal else (y, x)

    def set_limits(ax: plt.Axes) -> None:
        minimum = min(0, ohlc_df["Low"].min())  # include zero
        maximum = ohlc_df["High"].max()
        adjustment = (maximum - minimum) * 0.025
        limits = minimum - adjustment, maximum + adjustment
        if horizontal:
            ax.set_xlim(*limits)
        else:
            ax.set_ylim(*limits)

    # canvass
    fig, ax = plt.subplots()

    # sort out chart orientation
    good, bad = "darkblue", "darkorange"  # for colour blindness
    bar = ax.barh if horizontal else ax.bar
    reference = "left" if horizontal else "bottom"
    range = ohlc_df["High"] - ohlc_df["Low"]
    open_marker = "^" if horizontal else "<"
    close_marker = "v" if horizontal else ">"
    color = [
        good if open > close else bad
        for open, close in zip(ohlc_df.Open, ohlc_df.Close)
    ]

    # plot
    bar(
        ohlc_df.index,
        range,
        **{reference: ohlc_df["Low"]},
        color=color,
        linewidth=1.0,
        edgecolor="black",
        label="Range of prints through the 13 months",
        alpha=0.15,
    )
    ax.plot(
        *xy(ohlc_df["Open"], ohlc_df.index),
        marker=open_marker,
        linestyle="None",
        label="First print in the 13 months",
        color=good,
        markersize=5,
    )
    ax.plot(
        *xy(ohlc_df["Close"], ohlc_df.index),
        marker=close_marker,
        linestyle="None",
        label="Last print in the 13 months",
        color=bad,
        markersize=5,
    )
    ax.tick_params(axis="both", which="major", labelsize="x-small")
    set_limits(ax=ax)
    if not horizontal:
        ax.set_xticklabels(ohlc_df.index, rotation=90)
    finalise_plot(ax, **kwargs)

In [24]:
def get_lim(df):
    adjust = (df["High"].max() - df["Low"].min()) * 0.02
    return min(0, df["Low"].min()) - adjust, df["High"].max() + adjust

In [25]:
selected = [
    "Australia",
    "Canada",
    "China",
    "France",
    "Germany",
    "India",
    "Indonesia",
    "Italy",
    "Japan",
    "Korea",
    "United Kingdom",
    "United States",
    "New Zealand",
    "Norway",
    "Sweden",
    "Brazil",
    "Spain",
]
summary = get_recent_ohlc(annual_inflation[selected])

title = "Annual inflation prints over the most recent year"
lfooter = (
    "Year and month of latest print in the y-axis labels. "
    "Range is the 13 months up to and including the latest data. "
)
plot_ohlc(
    summary,
    horizontal=True,
    title=title,
    xlim=get_lim(summary),
    zero_y=True,
    xlabel="Per cent per year",
    tag="shorter",
    ylabel=None,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter=lfooter,
    show=SHOW,
)

In [26]:
summary = get_recent_ohlc(annual_inflation_raw)

# plot that dataset
plot_ohlc(
    summary,
    horizontal=False,
    title=title,
    ylabel="Per cent per year",
    ylim=get_lim(summary),
    tag="longer",
    xlabel=None,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter=lfooter,
    show=SHOW,
)

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_99555/49996124.py:62: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ohlc_df.index, rotation=90)


In [27]:
# Inflation trajectories post covid
ax = (
    annual_inflation_smooth[annual_inflation_smooth.index >= START_COVID]
    .drop(columns=EXCLUDE)
    .plot(alpha=0.5, linewidth=1)
)
ax.get_legend().remove()

mean_color, aus_color, lw = "black", "#dd0000", 3.5
mean = annual_inflation_smooth.dropna(axis=0, thresh=20).mean(axis=1)
mean[mean.index >= START_COVID].plot(lw=lw, color=mean_color, label="Mean")
(
    annual_inflation_smooth.loc[
        annual_inflation_smooth.index >= START_COVID, "Australia"
    ].plot(lw=lw, color=aus_color, label="Australia")
)
custom_lines = [
    Line2D([0], [0], color=mean_color, lw=lw),
    Line2D([0], [0], color=aus_color, lw=lw),
]
ax.legend(custom_lines, ["OECD monitored mean", "Australia"])

finalise_plot(
    ax,
    title="Annual inflation trajectories after COVID selected nations",
    xlabel=None,
    ylabel="Per cent per year",
    lfooter=f"{HENDERSON_TERM}-term Henderson moving averages over monthly data. "
    f'All OECD monitored nations except: {", ".join(EXCLUDE)}. ',
    rfooter=f"{SOURCE} CPI_PRICES",
    show=SHOW,
)

In [28]:
# months since annual inflation peak
HIGH_COVID = pd.Period("2021-01-01", freq="M")
results = pd.Series(dtype=int)
dataset = annual_inflation_raw[annual_inflation_raw.index >= HIGH_COVID]
also_exclude = EXCLUDE + ["Saudi Arabia"]
for nation in dataset.columns:
    if nation in also_exclude:
        continue
    series = dataset[nation]
    maximum = series.idxmax()
    last_valid = series.last_valid_index()
    elapsed = last_valid - maximum
    key = f"{nation} {str(last_valid.year)[2:]}-{last_valid.month:02}"
    results[key] = elapsed.n

# plot
results = results.iloc[np.lexsort([results.index, results.values])]
ax = results.plot.bar(width=0.8)
ax.tick_params(axis="both", which="major", labelsize="x-small")
finalise_plot(
    ax,
    title="Months since post-COVID annual inflation rate peak",
    xlabel=None,
    ylabel="Months",
    lfooter=(
        "Year and month of latest print in the y-axis labels. "
        # 'Using HMA trend data. '  # Only if using smoothed data
        f'Excluded: {", ".join(also_exclude)}. '
    ),
    rfooter=f"{SOURCE} CPI_PRICES",
    zero_y=True,
    show=SHOW,
)

In [29]:
# select >= 6 month since peak and starting below 3%
months, start = 6, 3

# smoothed or annual_inflation
series = annual_inflation

check = results[results >= months].index.to_list()
check = [c[:-months] for c in check]
check = [c for c in check if series.at[START_COVID, c] <= start]
print(check)
ax = series.loc[series.index >= START_COVID, check].plot(
    linewidth=2,
    **linestyle,
)
ax.legend(ncols=2, loc="best", fontsize="x-small")
finalise_plot(
    ax,
    title="Annual inflation trajectories selected nations",
    xlabel=None,
    ylabel="Per cent per year",
    tag="ahead-of-curve",
    # lfooter=f"{HENDERSON}-term Henderson moving averages. ",
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter=f"Selected on the basis that recent CPI peak >= {months} months ago. ",
    show=SHOW,
)

['Indonesia', 'Latvia', 'Lithuania', 'Netherlands', 'Costa Rica', 'Estonia', 'Switzerland', 'Korea', 'Slovenia', 'Spain', 'Canada', 'Greece', 'Luxembourg', 'New Zealand', 'United States']


In [30]:
# An anglosphere comparison
anglo = sorted([location_map[x] for x in chart_sets["anglosphere"]])

for data, tag, period in zip(
    [
        annual_inflation[anglo],
        annual_inflation_smooth[anglo],
        qrtly_inflation_annualised[anglo],
        qrtly_inflation_annualised_smooth[anglo],
    ],
    ["raw", "smooth", "raw", "smooth"],
    ["Annual", "Annual", "Quarterly annualised", "Quarterly annualised"],
):
    ax = data[data.index >= START_COVID].plot(
        linewidth=2,
        **linestyle,
    )
    ax.legend(ncols=2, loc="best", fontsize="x-small")
    lfooter = (
        f"{HENDERSON_TERM}-term Henderson moving averages. " if tag == "smooth" else ""
    )
    finalise_plot(
        ax,
        title=f"{period} inflation trajectories after COVID",
        xlabel=None,
        ylabel="Per cent per year",
        tag=f"angloshpere-{tag}",
        rfooter=f"{SOURCE} CPI_PRICES",
        lfooter=f"Anglosphere nations. {lfooter}",
        show=SHOW,
    )

### Quartrerly annualised - Australia vs OECD monitored mean

In [31]:
MIN_MEAN_REQUIRED = 25
for data, title, tag in zip(
    (
        annual_inflation,
        annual_inflation_smooth,
        qrtly_inflation_annualised,
        qrtly_inflation_annualised_smooth,
    ),
    ("Annual", "Annual", "Quarterly-annualised", "Quarterly-annualised"),
    ("annual", "annual_hma", "qrtly", "qrtly-hma"),
):
    footer = (
        "" if "hma" not in tag else f"{HENDERSON_TERM}-term Henderson moving average. "
    )

    frame = data.drop(columns=EXCLUDE)
    frame = frame[frame.index >= START_COVID]
    ax = frame.plot(lw=1, alpha=0.6)
    ax.get_legend().remove()

    mean_color, aus_color, lw = "black", "#dd0000", 3.5
    mean = frame.dropna(axis=0, thresh=MIN_MEAN_REQUIRED).mean(axis=1)
    mean.plot(lw=lw, color=mean_color, label="Mean")

    frame["Australia"].plot(lw=lw, color=aus_color, label="Australia")
    custom_lines = [
        Line2D([0], [0], color=mean_color, lw=lw),
        Line2D([0], [0], color=aus_color, lw=lw),
    ]
    ax.legend(custom_lines, ["OECD monitored mean", "Australia"], fontsize="small")

    ax.axhline(0, lw=0.75, color="#777777")
    finalise_plot(
        ax,
        title=f"{title} CPI inflation trajectories",
        xlabel=None,
        ylabel="Per cent per year",
        tag=f"Au-vs-OECD-{tag}",
        lfooter=footer + f'All OECD monitored nations except: {", ".join(EXCLUDE)}',
        rfooter=f"{SOURCE} CPI_PRICES",
        show=SHOW,
    )

### Chart by latest inflation status

In [32]:
LOWER, UPPER = 1.5, 3.5


def on_target(x):
    return (le(x, UPPER) & ge(x, LOWER)), "near target"


def too_high(x):
    return gt(x, UPPER), "too high"


def too_low(x):
    return lt(x, LOWER), "too low"

In [33]:
MIN_MEAN_REQUIRED = 25
for data, title, footer, tag in zip(
    (annual_inflation, qrtly_inflation_annualised, qrtly_inflation_annualised_smooth),
    ("Annual", "Quarterly-annualised", "Quarterly-annualised"),
    ("", "", f"{HENDERSON_TERM}-term Henderson moving average. "),
    ("annual", "qrtly", "qrtly-hma"),
):
    for comparator in (on_target, too_high, too_low):
        qx = data.drop(EXCLUDE, axis=1)
        renamer = {x: f"{x} ({(qx[x].ffill().iloc[-1]):0.1f}%)" for x in qx.columns}
        qx = qx.rename(columns=renamer)
        qx = qx[qx.index >= START_COVID]
        comparison, target_label = comparator(qx.ffill().iloc[-1])
        achievers = qx[qx.columns[comparison]].copy()

        ax = achievers.sort_index(axis=1).plot(
            linewidth=1,
            **linestyle,
        )

        ax.axhline(0, lw=0.75, color="#777777")
        ax.axhspan(
            LOWER,
            UPPER,
            color="#bbbbbb",
            zorder=-1,
            label=f"Target range ({LOWER}% - {UPPER}%)",
        )

        oecd_mean = qx.dropna(axis=0, thresh=MIN_MEAN_REQUIRED).mean(axis=1)
        oecd_mean.plot(
            lw=3.5,
            color="black",
            label=f"OECD Monitored Mean ({oecd_mean.iloc[-1]:0.1f}%)",
        )

        ax.legend(ncols=3, loc="best", fontsize="xx-small")
        finalise_plot(
            ax,
            title=f"{title} CPI growth {target_label}",
            xlabel=None,
            ylabel="Per cent per year",
            tag=f"Au-vs-achievers-{tag}",
            lfooter=f"{footer}End points included in legend. Excludes: {', '.join(EXCLUDE)}",  # "
            rfooter=f"{SOURCE} CPI_PRICES",
            show=SHOW,
        )

### Bringing in ABS Monthly CPI indicator

In [34]:
catalogue_id = "6484"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_ABS_meta_and_data(catalogue_id, verbose=False)
if abs_dict is None:
    sys.exit(-1)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Cache looks stale: Removing old cache version
We need to download this file from the ABS ...
Saving ABS download to cache.
Extracting DataFrames from the zip-file ...


In [35]:
MIN_MEAN_REQUIRED = 25
for service_type in ("Seasonally Adjusted", "Original"):
    # get ABS Australia data
    rows = find_rows(
        meta,
        {
            service_type: type_col,
            "All groups CPI(, s| ;)": did_col,
            ";  Australia": did_col,
            "Index": did_col,
        },
        regex=True,
        exact=False,
        verbose=False,
    )
    assert len(rows) == 1
    row = rows.iloc[0]

    id, units, did, table, series_type = iudts_from_row(row)
    aus_cpi = abs_dict[table][id]
    aus_cpi_change = aus_cpi.pct_change(3) * 100
    aus_cpi_annualised = annualise_percentages(aus_cpi_change, 4).dropna()

    for treatment in None, "smoothed":
        if treatment:
            aus_data = hma(aus_cpi_annualised, HENDERSON_TERM)
        else:
            aus_data = aus_cpi_annualised.copy()
        aus_data = aus_data[aus_data.index >= START_COVID]
        print(
            f"Australia {service_type} {treatment} endpoint: {aus_data.iloc[-1]:0.2f}%"
        )

        # plot all
        if treatment:
            frame = qrtly_inflation_annualised_smooth.drop(
                columns=EXCLUDE + ["Australia"]
            )
        else:
            frame = qrtly_inflation_annualised.drop(columns=EXCLUDE + ["Australia"])
        frame = frame[frame.index >= START_COVID]
        ax = frame.plot(lw=1, alpha=0.6)
        ax.get_legend().remove()

        mean_color, aus_color, lw = "black", "#dd0000", 3.5
        mean = frame.dropna(axis=0, thresh=MIN_MEAN_REQUIRED).mean(axis=1)
        mean.plot(lw=lw, color=mean_color, label="Mean")

        aus_data.plot(lw=lw, color=aus_color)
        custom_lines = [
            Line2D([0], [0], color=mean_color, lw=lw),
            Line2D([0], [0], color=aus_color, lw=lw),
        ]
        ax.legend(
            custom_lines,
            [
                "OECD monitored mean",
                f"Australia (ABS monthly CPI {series_type.lower()} series)",
            ],
            fontsize="small",
        )

        ax.axhline(0, lw=0.75, color="#777777")
        tag2 = "" if not treatment else treatment
        footer = (
            f"{HENDERSON_TERM}-term Henderson moving averages over monthly. "
            if treatment
            else ""
        )
        finalise_plot(
            ax,
            title="Quarterly-annualised inflation - Australia (ABS) v OECD",
            xlabel=None,
            ylabel="Per cent per year",
            tag=f"Au-from-ABS-{service_type.replace(' ', '-')}-vs-OECD-{tag2}",
            lfooter=f'{footer}All OECD monitored nations except: {", ".join(EXCLUDE)}',
            rfooter=f"{SOURCE} CPI_PRICES. {source} T{table}",
            show=SHOW,
        )

/Users/bryanpalmer/ABS/notebooks/abs_data_capture.py:689: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  meta_select[column].str.contains(phrase, regex=regex)


Australia Seasonally Adjusted None endpoint: 5.96%
Australia Seasonally Adjusted smoothed endpoint: 7.90%


/Users/bryanpalmer/ABS/notebooks/abs_data_capture.py:689: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  meta_select[column].str.contains(phrase, regex=regex)


Australia Original None endpoint: 5.26%
Australia Original smoothed endpoint: 7.79%


### Latest CPI components

In [36]:
# set_up
all_items = ("CP18ALTT", "CPALTT01")
food = ("CP180100", "CP010000")
energy = ("CP18GREN", "CPGREN01")
points = {
    "All items": all_items,
    "Food and non-alcoholic beverages": food,
    "Energy": energy,
}
locations = list(location_map.keys())
start = TODAY.year - 2
measure = "IXOB"

In [37]:
def get_data_date(series):
    return str(pd.Period(series.last_valid_index(), freq="M"))


frame = pd.DataFrame()
loop_count = 0
for component, subjects in points.items():
    cpi_x = download_CPI_data(start=start, subject_tuple=subjects)
    inf_x = (
        cpi_x.pct_change(12, fill_method=None)  # Note: default fill_method is nasty
        * 100  # per cent
    )
    if not loop_count:
        latest_date = inf_x.apply(get_data_date)
        renamer = {x: f"{x} {str(latest_date[x])[2:]}" for x in inf_x.columns}
    latest = inf_x.ffill().iloc[-1].rename(renamer)
    if not loop_count:
        latest = latest.sort_values()
    frame[component] = latest
    loop_count += 1

# frame

HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP18ALTT.IXOB.Q/all?startTime=2021
HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP180100.IXOB.Q/all?startTime=2021
HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP18GREN.IXOB.Q/all?startTime=2021


In [38]:
# ... and plot
sns.set(font_scale=1)
ax = sns.scatterplot(frame)
labels = ax.get_xticklabels()
ax.set_xticklabels(
    labels=labels,
    rotation=90,
)
ax.legend(loc="upper left")

finalise_plot(
    ax,
    title="Latest CPI growth - selected nations and items",
    ylabel="Per cent over year",
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter="Dates for latest headline (All items) data in YY-MM format",
    show=SHOW,
)

# tidy-up
plt.style.use("fivethirtyeight")

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_99555/1357312145.py:5: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(


## Real interest rates

### Get Bank for International Settlements (BIS) Data

In [39]:
bis_meta, bis_data = get_bis_cbpr()

### Notional real interest rates

In [40]:
euro_zone = {
    "Austria",
    "Belgium",
    "Croatia",
    "Cyprus",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Ireland",
    "Italy",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Portugal",
    "Slovakia",
    "Slovenia",
    "Spain",
}

recent = pd.Period("2005-01", freq="M")
inflation_r = annual_inflation[annual_inflation.index >= recent].copy()
inflation_r = inflation_r.rename(columns=location_map)
cb_policy = bis_data.resample(rule="M", kind="period").mean()
cb_policy = cb_policy[cb_policy.index >= recent]

for inf_nation in inflation_r.sort_index(axis=1):
    cbp_nation = inf_nation if inf_nation in cb_policy.columns else None
    if not cbp_nation and inf_nation in euro_zone:
        cbp_nation = "Euro area"
    if not cbp_nation:
        continue

    real_rates = cb_policy[cbp_nation] - inflation_r[inf_nation]
    ax = real_rates.plot(lw=2)
    ax.axhline(0, c="#555555", lw=0.5)
    title = f"Notional Real Interest Rates for {inf_nation}"
    finalise_plot(
        ax,
        title=title,
        ylabel="Rate (Per cent per year)",
        rfooter="Source: OECD, BIS",
        lfooter="Real Interest Rate = Central Bank Policy Rate - Inflation Rate. "
        "Expansionary policy < 0. Contractionary policy > 0.",
        show=SHOW,
    )

## Finished

In [41]:
%reload_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Apr 22 2023 10:08:29

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.12.0

sys       : 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]
numpy     : 1.24.2
pandas    : 2.0.0
matplotlib: 3.7.1
seaborn   : 0.12.2

Watermark: 2.3.1



In [42]:
print("Done")

Done
